# Experiment - Prompting OpenAI API

## Prepare Notebook

In [1]:
import os
import re
import sys
import logging
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

if False:
    %pip install yfinance==0.2.43
    %pip install openai==1.51.2

%load_ext dotenv

FUNDAMENTALS_PATH = os.getenv("FUNDAMENTALS_PATH")
LLM_PROMPTS_PATH = os.getenv("LLM_PROMPTS_PATH")
FUNDAMENTALS_PATH = os.getenv("FUNDAMENTALS_PATH")
HISTORIC_PATH = os.getenv("HISTORIC_PATH")
MACRO_PATH = os.getenv("MACRO_PATH")
US_ECON_PATH = f"{MACRO_PATH}/us"
OPTIONS_PATH = os.getenv("OPTIONS_PATH")
LOGS_PATH = os.getenv("LOGS_PATH")
NEWS_PATH = os.getenv("NEWS_PATH")
paths = [LOGS_PATH]
for path in paths:
    if path and not os.path.exists(path):
        os.makedirs(path)

module_path = os.path.abspath(os.path.join(os.getcwd(), 'utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    logging.info("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/drl-dataset-quant"
    FUNDAMENTALS_PATH = DATA_PATH + FUNDAMENTALS_PATH
    HISTORIC_PATH = DATA_PATH + HISTORIC_PATH
    MACRO_PATH = DATA_PATH + MACRO_PATH
    OPTIONS_PATH = DATA_PATH + OPTIONS_PATH
    NEWS_PATH = DATA_PATH + NEWS_PATH
    sys.path.insert(1, "/kaggle/usr/lib/utils")

In [2]:
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from data_utils import get_fundamentals, get_historic, get_macro_data, feature_engineer, get_historic_optionschain_summary, get_economic_indicators

## Environment and Constants

In [3]:
START_DATE = '20120101'
END_DATE = '20200101'
TARGET = 'META'

# Data Wrangling

### Fundamental Data

Scraped from the SEC and Macro Trends.

In [4]:
fundamentals = get_fundamentals(TARGET, FUNDAMENTALS_PATH)
if fundamentals.isna().any().any():
    raise ValueError(f"Missing values detected in fundamentals data for {TARGET}. Fix the data before saving.")
fundamentals.head(3)


,Net Income,Invested Capital,Return on Investment,Current Ratio,Long-term Debt / Capital,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,...,Net Income_QoQ_Growth,Net Income_YoY_Growth,Free Cash Flow Per Share_QoQ_Growth,Free Cash Flow Per Share_YoY_Growth,Operating Cash Flow Per Share_QoQ_Growth,Operating Cash Flow Per Share_YoY_Growth,Return on Equity_QoQ_Growth,Return on Equity_YoY_Growth,Return on Assets_QoQ_Growth,Return on Assets_YoY_Growth
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-31 00:00:00+00:00,1.030000e+09,2.410000e+09,1.7131,5.7738,0.1451,75.0253,52.2796,52.2796,59.3212,51.0638,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2011-03-31 00:00:00+00:00,1.260000e+09,2.410000e+09,2.0879,5.1212,0.0751,76.8257,47.3188,47.3188,56.0226,45.6750,...,0.223301,0.0,0.402151,0.0,0.221332,0.0,0.095701,0.0,0.095720,0.0
2011-06-30 00:00:00+00:00,1.450000e+09,2.410000e+09,2.4013,5.1212,0.0751,76.8257,47.3188,47.3188,56.0226,45.6750,...,0.150794,0.0,0.402151,0.0,0.221332,0.0,0.045177,0.0,0.045142,0.0


### Historic Price

Procured from IBKR API.

In [5]:
stock_data = get_historic(TARGET, HISTORIC_PATH)
if stock_data.isna().any().any():
    raise ValueError(f"Missing values detected in stock_data for {TARGET}. Fix the data before saving.")

stock_data.tail(3)

,Open,High,Low,Close,Volume,Average,Barcount,IV_Open,IV_High,IV_Low,...,IV_Average,IV_Barcount,HV_Open,HV_High,HV_Low,HV_Close,HV_Volume,HV_Average,HV_Barcount,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-18 00:00:00+00:00,620.91,627.27,596.38,597.19,1490586,9926,614.236,0.300949,0.337413,0.293805,...,0.0,0.337413,0.277248,0.277248,0.277248,0.277248,1.0,0.0,0.277248,META
2024-12-19 00:00:00+00:00,610.20,611.52,595.00,595.57,1280265,8615,603.473,0.343747,0.347223,0.323126,...,0.0,0.347223,0.276232,0.276232,0.276232,0.276232,1.0,0.0,0.276232,META
2024-12-20 00:00:00+00:00,590.04,603.11,584.00,585.25,4112358,13407,590.407,0.353605,0.359161,0.309093,...,0.0,0.359161,0.282534,0.282534,0.282534,0.282534,1.0,0.0,0.282534,META


### Historic Economic Date

Also procured from FRED, BLS, and ISM.

In [6]:
econ_df = get_economic_indicators(US_ECON_PATH)
print(econ_df.columns)
econ_df.head(3)


Index(['PMI', 'PPI', 'Retail Sales', 'Employment', 'Yield_Curve',
       'Treasury Yields', 'CPI', 'Consumer Confidence', 'Consumer_Confidence',
       'Housing Starts', 'Housing_Starts', 'M2_Money_Supply', 'Retail_Sales',
       'Treasury_Yields', 'GDP', 'Yield Curve', 'PMI_YoY', 'PPI_YoY',
       'Retail Sales_YoY', 'Employment_YoY', 'Yield_Curve_YoY',
       'Treasury Yields_YoY', 'CPI_YoY', 'Consumer Confidence_YoY',
       'Consumer_Confidence_YoY', 'Housing Starts_YoY', 'Housing_Starts_YoY',
       'M2_Money_Supply_YoY', 'Retail_Sales_YoY', 'Treasury_Yields_YoY',
       'GDP_YoY', 'Yield Curve_YoY', 'PMI_QoQ', 'PPI_QoQ', 'Retail Sales_QoQ',
       'Employment_QoQ', 'Yield_Curve_QoQ', 'Treasury Yields_QoQ', 'CPI_QoQ',
       'Consumer Confidence_QoQ', 'Consumer_Confidence_QoQ',
       'Housing Starts_QoQ', 'Housing_Starts_QoQ', 'M2_Money_Supply_QoQ',
       'Retail_Sales_QoQ', 'Treasury_Yields_QoQ', 'GDP_QoQ',
       'Yield Curve_QoQ'],
      dtype='object')


,PMI,PPI,Retail Sales,Employment,Yield_Curve,Treasury Yields,CPI,Consumer Confidence,Consumer_Confidence,Housing Starts,...,CPI_QoQ,Consumer Confidence_QoQ,Consumer_Confidence_QoQ,Housing Starts_QoQ,Housing_Starts_QoQ,M2_Money_Supply_QoQ,Retail_Sales_QoQ,Treasury_Yields_QoQ,GDP_QoQ,Yield Curve_QoQ
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00+00:00,52.9,181.9,314557.0,129795.0,2.81,3.73,217.488,98.50824,98.50824,614.0,...,-0.000391,0.00032,0.00032,0.118893,0.118893,0.009094,0.132809,0.032172,0.014601,-0.032028
2010-02-01 00:00:00+00:00,52.9,181.0,310925.0,129702.0,2.80,3.69,217.281,98.56321,98.56321,604.0,...,-0.000391,0.00032,0.00032,0.118893,0.118893,0.009094,0.132809,0.032172,0.014601,-0.032028
2010-03-01 00:00:00+00:00,52.9,183.3,360679.0,129865.0,2.82,3.73,217.353,98.56012,98.56012,636.0,...,-0.000391,0.00032,0.00032,0.118893,0.118893,0.009094,0.132809,0.032172,0.014601,-0.032028


### Historic Macro Indices

Also procured from IBKR API, FRED, BLS, and ISM.

In [7]:
macro_data = get_macro_data(MACRO_PATH)
if macro_data.isna().any().any():
    raise ValueError(f"Missing values detected in macro_data for {TARGET}. Fix the data before saving.")

macro_data.tail(3)


,SPX_Open,SPX_High,SPX_Low,SPX_Close,SPX_Volume,SPX_Average,SPX_Barcount,SPX_IV_Open,SPX_IV_High,SPX_IV_Low,...,IRX_Volume,IRX_Average,IRX_Barcount,IRX_HV_Open,IRX_HV_High,IRX_HV_Low,IRX_HV_Close,IRX_HV_Volume,IRX_HV_Average,IRX_HV_Barcount
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-28 00:00:00+00:00,3750.01,3756.12,3723.31,3727.04,0.0,0.0,21084.0,0.164000,0.181684,0.163412,...,0.0,0.0,1.0,1.423959,1.423959,1.423959,1.423959,1.0,1.423959,0.0
2020-12-29 00:00:00+00:00,3736.19,3744.63,3730.21,3732.04,0.0,0.0,20553.0,0.175493,0.180779,0.172350,...,0.0,0.0,13.0,1.438135,1.438135,1.438135,1.438135,1.0,1.438135,0.0
2020-12-30 00:00:00+00:00,3733.27,3760.20,3726.88,3756.07,0.0,0.0,20639.0,0.180144,0.181874,0.173461,...,0.0,0.0,13.0,1.413990,1.413990,1.413990,1.413990,1.0,1.413990,0.0


### Options Data

Bought from iVolatility.

In [8]:
optionchain_data = get_historic_optionschain_summary(TARGET, OPTIONS_PATH)
if optionchain_data.isna().any().any():
    raise ValueError(f"Missing values detected in optionchain_data for {TARGET}. Fix the data before saving.")
optionchain_data.tail(3)


,OTM_IV_Call,expiration_date,OTM_IV_Put,OTM_Skew,ATM_IV_Call,ATM_IV_Put,ATM_Skew,ITM_IV_Call,ITM_IV_Put,ITM_Skew,Skew,Vol_Surface,DTE
Date,,,,,,,,,,,,,
2024-12-18 00:00:00+00:00,0.366653,2025-01-10,0.319769,-0.127870,0.359481,0.311878,-0.132421,0.364192,0.325368,-0.106603,0.867579,FLAT,23
2024-12-19 00:00:00+00:00,0.337653,2025-01-10,0.347327,0.028651,0.341921,0.327597,-0.041893,0.338562,0.339205,0.001899,0.958107,SMILE,22
2024-12-20 00:00:00+00:00,0.308769,2025-01-10,0.292158,-0.053797,0.307702,0.271117,-0.118898,0.307549,0.333964,0.085889,0.881102,SMILE,21


# Feature Engineering

Merge data and create price features.

In [9]:
fundamentals['Quarter'] = pd.to_datetime(fundamentals.index, utc=True).to_period('Q')
stock_data['Quarter'] = pd.to_datetime(stock_data.index, utc=True).to_period('Q')

stk_idx = stock_data.index
stock_aug_data = pd.merge(stock_data, fundamentals, on='Quarter', how='left')
stock_aug_data = stock_aug_data.set_index(stk_idx, drop=False)
stock_aug_data = pd.merge_asof(
    stock_aug_data.sort_index(),
    econ_df.sort_index(),
    left_index=True,
    right_index=True,
    direction='backward'
).bfill().ffill()
if stock_aug_data.isna().any().any():
    raise ValueError(f"Missing macros detected in stock_aug_data for {TARGET}.")
stock_aug_data = pd.merge(
    stock_aug_data,
    macro_data,
    left_index=True,
    right_index=True,
    how='left'
).bfill().ffill()
stock_aug_data = pd.merge(
    stock_aug_data,
    optionchain_data,
    left_index=True,
    right_index=True,
    how='left'
).bfill().ffill()
stock_aug_data = stock_aug_data.drop(columns=[
    col for col in stock_aug_data.columns if col.endswith("_Average") or col.endswith("_Barcount") or col in ["Average", "Barcount"]
])

if stock_aug_data.isna().any().any():
    raise ValueError(f"Missing values detected in stock_aug_data for {TARGET}.")

stock_aug_data.tail(3)

,Open,High,Low,Close,Volume,IV_Open,IV_High,IV_Low,IV_Close,IV_Volume,...,OTM_Skew,ATM_IV_Call,ATM_IV_Put,ATM_Skew,ITM_IV_Call,ITM_IV_Put,ITM_Skew,Skew,Vol_Surface,DTE
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-18 00:00:00+00:00,620.91,627.27,596.38,597.19,1490586,0.300949,0.337413,0.293805,0.337413,1.0,...,-0.127870,0.359481,0.311878,-0.132421,0.364192,0.325368,-0.106603,0.867579,FLAT,23.0
2024-12-19 00:00:00+00:00,610.20,611.52,595.00,595.57,1280265,0.343747,0.347223,0.323126,0.342461,1.0,...,0.028651,0.341921,0.327597,-0.041893,0.338562,0.339205,0.001899,0.958107,SMILE,22.0
2024-12-20 00:00:00+00:00,590.04,603.11,584.00,585.25,4112358,0.353605,0.359161,0.309093,0.314236,1.0,...,-0.053797,0.307702,0.271117,-0.118898,0.307549,0.333964,0.085889,0.881102,SMILE,21.0


In [10]:
engineered_df = feature_engineer(stock_aug_data.copy(), market_ticker='SPX', roll_window=5)
engineered_df.reset_index(inplace=True, drop=False)
for col in engineered_df.columns:
    if hasattr(engineered_df[col], "dtype") and (isinstance(engineered_df[col].dtype, pd.PeriodDtype) or engineered_df[col].dtype == 'object'):
        engineered_df[col] = engineered_df[col].astype(str)

if engineered_df.isna().any().any():
    raise ValueError(f"Missing engineered values detected in engineered_df for {TARGET}.")


# Combine News

In [11]:
from pathlib import Path

news_df = pd.read_parquet(f'{NEWS_PATH}/{TARGET}_2012-01-01_2025-01-21.parquet')
news_df['datetime'] = pd.to_datetime(news_df['datetime'], utc=True).dt.normalize()
news_df['content'] = news_df['content'].fillna('').str.strip()
news_df = news_df.groupby('datetime')['content'].apply(list).reset_index()
news_df['content'] = news_df['content'].apply(lambda x: x if x else [])

missing_csv_path = Path(f'{NEWS_PATH}/missing_{TARGET}.csv')
if missing_csv_path.exists():
    missing_df = pd.read_csv(missing_csv_path)
    missing_df['datetime'] = pd.to_datetime(missing_df['datetime'], utc=True).dt.normalize()
    missing_df['content'] = missing_df['content'].fillna('').str.strip()
    missing_df['summary'] = missing_df['summary'].fillna('').str.strip()
    missing_df['content'] = missing_df.apply(
        lambda row: row['content'] if row['content'] else row['summary'], axis=1
    )
    missing_grouped = missing_df.groupby('datetime')['content'].apply(list).reset_index()
    missing_grouped['content'] = missing_grouped['content'].apply(lambda x: x if x else [])

    news_df = pd.concat([news_df, missing_grouped], ignore_index=True)
    news_df = news_df.groupby('datetime')['content'].sum().reset_index()

engineered_df = pd.merge(engineered_df, news_df, left_on='Date', right_on='datetime', how='left')
engineered_df = engineered_df[~engineered_df.index.duplicated(keep='first')].drop('datetime', axis=1)
engineered_df = engineered_df.loc[:, ~engineered_df.columns.duplicated(keep='first')]

engineered_df.tail(3)

,Date,Open,High,Low,Close,Volume,IV_Open,IV_High,IV_Low,IV_Close,...,Volume_Weighted_Returns,BB_Upper,BB_Middle,BB_Lower,BB_Width,IV_Percentile,VIX_Impact,Momentum_Long,Momentum_Short,content
3166,2024-12-18 00:00:00+00:00,620.91,627.27,596.38,597.19,1490586,0.300949,0.337413,0.293805,0.337413,...,-13787.401583,641.071202,618.402,595.732798,0.075920,1.00000,0.0,False,True,[<p>Entrepreneur and investor <strong>Mark Cub...
3167,2024-12-19 00:00:00+00:00,610.20,611.52,595.00,595.57,1280265,0.343747,0.347223,0.323126,0.342461,...,-14045.018405,636.049924,611.358,586.666076,0.082919,1.00000,0.0,False,True,"[<p>Meta Platforms (NASDAQ:<a class=""ticker"" h..."
3168,2024-12-20 00:00:00+00:00,590.04,603.11,584.00,585.25,4112358,0.353605,0.359161,0.309093,0.314236,...,-25873.776216,634.224231,604.338,574.451769,0.102132,0.49069,0.0,False,True,[<p>Benzinga readers pick which Magnificent 7 ...


In [12]:
engineered_df = pl.from_pandas(engineered_df)
engineered_df = engineered_df.with_columns(
        pl.col("content").cast(pl.List(pl.Utf8))
    )
engineered_df = engineered_df.unique()
engineered_df = engineered_df.sort(by='Date')
output_file = f"{HISTORIC_PATH}/engineered_{TARGET}_data.parquet"
engineered_df.write_parquet(output_file)

# Persist to Parquet

Note macro and economic is downloaded once.

In [13]:
tickers = ["META", "AAPL", "MSFT", "GOOGL", "TSLA", "AMZN"]
for ticker in tqdm(tickers):
    fundamentals = get_fundamentals(ticker, FUNDAMENTALS_PATH)
    stock_data = get_historic(ticker, HISTORIC_PATH)
    optionchain_data = get_historic_optionschain_summary(ticker, OPTIONS_PATH)

    fundamentals['Quarter'] = pd.to_datetime(fundamentals.index, utc=True).to_period('Q')
    stock_data['Quarter'] = pd.to_datetime(stock_data.index, utc=True).to_period('Q')

    stk_idx = stock_data.index
    stock_aug_data = pd.merge(stock_data, fundamentals, on='Quarter', how='left')
    stock_aug_data = stock_aug_data.set_index(stk_idx, drop=False)
    stock_aug_data = pd.merge_asof(
        stock_aug_data.sort_index(),
        econ_df.sort_index(),
        left_index=True,
        right_index=True,
        direction='backward'
    ).bfill().ffill()
    if stock_aug_data.isna().any().any():
        raise ValueError(f"Missing macros detected in stock_aug_data for {ticker}.")
    stock_aug_data = pd.merge(
        stock_aug_data,
        macro_data,
        left_index=True,
        right_index=True,
        how='left'
    ).bfill().ffill()
    stock_aug_data = pd.merge(
        stock_aug_data,
        optionchain_data,
        left_index=True,
        right_index=True,
        how='left'
    ).bfill().ffill()
    stock_aug_data = stock_aug_data.drop(columns=[
        col for col in stock_aug_data.columns if col.endswith("_Average") or col.endswith("_Barcount") or col in ["Average", "Barcount"]
    ])
    if stock_aug_data.isna().any().any():
        raise ValueError(f"Missing values detected in stock_aug_data for {ticker}.")

    stock_aug_data.tail(3)
    engineered_df = feature_engineer(stock_aug_data.copy(), market_ticker='SPX', roll_window=5)
    engineered_df.reset_index(inplace=True, drop=False)
    for col in engineered_df.columns:
        if hasattr(engineered_df[col], "dtype") and (isinstance(engineered_df[col].dtype, pd.PeriodDtype) or engineered_df[col].dtype == 'object'):
            engineered_df[col] = engineered_df[col].astype(str)

    if engineered_df.isna().any().any():
        raise ValueError(f"Missing values detected in engineered data for {ticker}.")

    news_df = pd.read_parquet(f'{NEWS_PATH}/{ticker}_2012-01-01_2025-01-21.parquet')
    news_df['datetime'] = pd.to_datetime(news_df['datetime'], utc=True).dt.normalize()
    news_df['content'] = news_df['content'].fillna('').str.strip()
    news_df = news_df.groupby('datetime')['content'].apply(list).reset_index()
    news_df['content'] = news_df['content'].apply(lambda x: x if x else [])

    missing_csv_path = Path(f'{NEWS_PATH}/missing_{ticker}.csv')
    if missing_csv_path.exists():
        missing_df = pd.read_csv(missing_csv_path)
        missing_df['datetime'] = pd.to_datetime(missing_df['datetime'], utc=True).dt.normalize()
        missing_df['content'] = missing_df['content'].fillna('').str.strip()
        missing_df['summary'] = missing_df['summary'].fillna('').str.strip()
        missing_df['content'] = missing_df.apply(
            lambda row: row['content'] if row['content'] else row['summary'], axis=1
        )
        missing_grouped = missing_df.groupby('datetime')['content'].apply(list).reset_index()
        missing_grouped['content'] = missing_grouped['content'].apply(lambda x: x if x else [])
        news_df = pd.concat([news_df, missing_grouped], ignore_index=True)
        news_df = news_df.groupby('datetime')['content'].sum().reset_index()

    engineered_df = pd.merge(engineered_df, news_df, left_on='Date', right_on='datetime', how='left')
    # Front fill missing news days.
    engineered_df['content'] = engineered_df['content'].ffill().apply(
        lambda x: x if isinstance(x, list) else []
    )
    engineered_df = engineered_df[~engineered_df.index.duplicated(keep='first')].drop('datetime', axis=1)
    engineered_df = engineered_df.loc[:, ~engineered_df.columns.duplicated(keep='first')]
    engineered_df = pl.from_pandas(engineered_df)
    engineered_df = engineered_df.unique()
    engineered_df = engineered_df.sort(by='Date')

    output_file = f"{HISTORIC_PATH}/engineered_{ticker}_data.parquet"
    engineered_df.write_parquet(output_file)

    print(f"Parquet written to {output_file}")


  0%|          | 0/6 [00:00<?, ?it/s]

Parquet written to ./data/historic/engineered_META_data.parquet
Parquet written to ./data/historic/engineered_AAPL_data.parquet
Parquet written to ./data/historic/engineered_MSFT_data.parquet
Parquet written to ./data/historic/engineered_GOOGL_data.parquet
Parquet written to ./data/historic/engineered_TSLA_data.parquet
Parquet written to ./data/historic/engineered_AMZN_data.parquet
